In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.metrics import average_precision_score, f1_score,confusion_matrix
from sklearn import manifold
from xgboost.sklearn import XGBClassifier
from xgboost import plot_importance, to_graphviz
import lightgbm as lgb 
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

#from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv('../input/PS_20174392719_1491204439457_log.csv')
#df = pd.read_csv('./input/PS_20174392719_1491204439457_log.csv.zip',compression='zip')

df = df.rename(columns={'oldbalanceOrg':'oldBalanceOrig', 'newbalanceOrig':'newBalanceOrig', \
                        'oldbalanceDest':'oldBalanceDest', 'newbalanceDest':'newBalanceDest'})


In [ ]:
df[:1]

In [ ]:
df.describe()

In [ ]:
df.groupby('type')['isFraud','isFlaggedFraud'].sum()

In [ ]:
df.groupby('type')['amount'].mean()

In [ ]:
%%time
# Start Feature extraction
# Merchant flag for source and dist
# Trans tye for fraud trans type
data = df.copy()
# add source and target type
data['OrigC']=data['nameOrig'].apply(lambda x: 1 if str(x).find('C')==0 else 0)
data['DestC']=data['nameDest'].apply(lambda x: 1 if str(x).find('C')==0 else 0)
data['TRANSFER']=data['type'].apply(lambda x: 1 if x=='TRANSFER' else 0)
data['CASH_OUT']=data['type'].apply(lambda x: 1 if x=='CASH_OUT' else 0)

#

In [ ]:
# Trans Amount from Loan ? killed features from  Arjun Joshua
data['OrigAmntErr']=(abs(data.oldBalanceOrig-data.newBalanceOrig)-data.amount)*data['OrigC']
data['DestAmntErr']=(abs(data.oldBalanceDest-data.oldBalanceDest)-data.amount)*data['DestC']

In [ ]:
def Fplot(DF,catlist):
    nf = DF.shape[1]
    frdf=DF[catlist][DF.isFraud == 1]
    cldf=DF[catlist][DF.isFraud == 0]
    sns.set(font_scale=1)
    for i, cn in enumerate(catlist):
        if cn != 'isFraud':
            plt.figure(figsize=(12,nf*8))
            gs = gridspec.GridSpec(nf*2, 1)
            ax = plt.subplot(gs[i])
            ax.set_title(str(cn)+'(Fraud-Red/Normal-Green)')
            sns.distplot(frdf[cn], bins=50, color='Red', hist=True)
            ax = plt.subplot(gs[i+1])
            sns.distplot(cldf[cn], bins=50, color='Green', hist=True)
            DF[[cn,'isFraud']].boxplot(by='isFraud', vert=False,figsize=(12,5))
            plt.show()
    

In [ ]:
dt=data.sample(n=500000)
flist=['step', u'amount','oldBalanceOrig',
       'newBalanceOrig', 'oldBalanceDest', 'newBalanceDest',
       'isFlaggedFraud', 'OrigC', 'DestC', 'TRANSFER',
       'CASH_OUT', 'OrigAmntErr', 'DestAmntErr']
Fplot(dt,flist)

In [ ]:
# drop list 
droplist=['isFlaggedFraud','OrigC','type','nameDest','nameOrig']

In [ ]:
#print XGBoost result
def presult(clf,x_test,y_test, plotimp=1):
    y_prob=clf.predict_proba(x_test)
    y_pred=clf.predict(x_test)
    print ('AUPRC :', (average_precision_score(y_test, y_prob[:, 1])))
    print ('F1 - macro :',(f1_score(y_test,y_pred,average='macro')))
    print  ('Confusion_matrix : ')
    print (confusion_matrix(y_test,y_pred))
    sns.set(font_scale=1.5)
    #sns.heatmap(confusion_matrix(testY,y_pred), annot=True,annot_kws={"size": 15},fmt='10g')
    #plt.show()
    if plotimp==1:
        plot_importance(clf)
        plt.show()

In [ ]:
#Train/Test Split

In [ ]:
MLData=data.drop(labels=droplist,axis=1)
X=MLData.drop('isFraud',axis=1)
Y=MLData.isFraud


In [ ]:
trainX, testX, trainY, testY = train_test_split(X, Y, test_size = 0.3,random_state=42)

In [ ]:
print ('trainX.step.min,max,count', trainX.step.min(),trainX.step.max(),trainX.step.count())
print ('testX.step.min,max,count',testX.step.min(),testX.step.max(),testX.step.count())
print ('y=1 total, train, test',sum(Y),sum(trainY),sum(testY))

DateTime based feature "step" have same value in train and test data. Below i use shuffle=False  to not know the future when learning

In [ ]:
trainX, testX, trainY, testY = train_test_split(X, Y, test_size = 0.3,random_state=42, shuffle=False)

In [ ]:
print ('trainX.step.min,max,count', trainX.step.min(),trainX.step.max(),trainX.step.count())
print ('testX.step.min,max,count',testX.step.min(),testX.step.max(),testX.step.count())
print ('y=1 total, train, test',sum(Y),sum(trainY),sum(testY))

In [ ]:
%%time
#Base metrics 
weights = (Y == 0).sum() / (1.0 * (Y == 1).sum())
clf = XGBClassifier( scale_pos_weight = weights, n_jobs = 4, random_state=42)
clf.fit(trainX, trainY)
print ('Test')
presult(clf,testX,testY,1)

In [ ]:
#add DestC to drop list
droplist=['isFlaggedFraud','OrigC','type','nameDest','nameOrig','DestC']

In [ ]:
%%time
# Check series transaction for same client
dt=data.copy()
dt = dt.sort_values(['step'])
print (sum(dt.groupby('nameOrig')['step'].count()>1)) #9298 ±0.2% 
# Only ±10 000 transaction for same client. Impossible create client profile features :-( 
#=> We can drop Trans type WO Fraud case




In [ ]:
# drop Trans type WO Fraud
MLData=data.copy()
#only TRANSFER and CASH_OUT
MLData=MLData.loc[(MLData.TRANSFER+MLData.CASH_OUT)>0]
MLData.drop(droplist,axis=1,inplace=True)
#add CASH_OUT to drop list
MLData=MLData.drop(['CASH_OUT'],axis=1)
X=MLData.drop('isFraud',axis=1)
Y=MLData.isFraud
trainX, testX, trainY, testY = train_test_split(X, Y, test_size = 0.3, shuffle=False)

In [ ]:
%%time
# metrics only for CASH OUT and transfer in train
weights = (Y == 0).sum() / (1.0 * (Y == 1).sum())
clf = XGBClassifier( scale_pos_weight = weights, n_jobs = 4, random_state=42)
clf.fit(trainX, trainY)
presult(clf,testX,testY,1)

DestAmntErr - drop candidate

In [ ]:
# modify step to hours in day. 
X['step24']=X.step%24

In [ ]:
X.step24.describe()

In [ ]:
trainX, testX, trainY, testY = train_test_split(X, Y, test_size = 0.3, shuffle=False)

In [ ]:
%%time
# metrics step in hour(s)
weights = (Y == 0).sum() / (1.0 * (Y == 1).sum())
clf = XGBClassifier( scale_pos_weight = weights, n_jobs = 4, random_state=42)
clf.fit(trainX, trainY)
presult(clf,testX,testY,1)

In [ ]:
#  drop DestAmntErr and update newBalanceDest
X.loc[(X.DestAmntErr != 0) & (X.newBalanceDest == 0),'newBalanceDest'] = -1
X=X.drop('DestAmntErr',axis=1)
MLData.loc[(MLData.DestAmntErr != 0) & (MLData.newBalanceDest == 0),'newBalanceDest'] = -1
MLData=MLData.drop('DestAmntErr',axis=1)

In [ ]:
X.newBalanceDest.describe()

In [ ]:
trainX, testX, trainY, testY = train_test_split(X, Y, test_size = 0.3, shuffle=False)

In [ ]:
%%time
# metrics step in hour(s)
weights = (Y == 0).sum() / (1.0 * (Y == 1).sum())
clf = XGBClassifier( scale_pos_weight = weights, n_jobs = 4, random_state=42)
clf.fit(trainX, trainY)
presult(clf,testX,testY,1)

In [ ]:
%%time
#LGBMClassifier
# metrics for 0.3 test size
clf = lgb.LGBMClassifier(n_estimators=100,max_depth=3, n_jobs = 4, random_state=42)
clf.fit(trainX, trainY)
presult(clf,testX,testY,0)

It perfect but makes impossible improve the ML model 

Try 0.5 test size

In [ ]:
trainX, testX, trainY, testY = train_test_split(X, Y, test_size = 0.5, shuffle=False)

In [ ]:
%%time
# metrics for 0.5 test size
weights = (Y == 0).sum() / (1.0 * (Y == 1).sum())
clf = XGBClassifier( scale_pos_weight = weights, n_jobs = 4, random_state=42)
clf.fit(trainX, trainY)
presult(clf,testX,testY,0)

In [ ]:
#ok try next step

In [ ]:
#in first scale data. Theory - result do not change for random forest (XGB)

In [ ]:
%%time
scaler = StandardScaler()
scaler.fit(trainX)
# metrics for 0.5 test size
weights = (Y == 0).sum() / (1.0 * (Y == 1).sum())
clf = XGBClassifier( scale_pos_weight = weights, n_jobs = 4, random_state=42)
clf.fit(scaler.transform(trainX), trainY)
presult(clf,scaler.transform(testX),testY,0)

In [ ]:
%%time
#LGBMClassifier
scaler = StandardScaler()
scaler.fit(trainX)
# metrics for 0.3 test size
clf = lgb.LGBMClassifier(n_estimators=100,max_depth=3, n_jobs = 4, random_state=42)
clf.fit(scaler.transform(trainX), trainY)
presult(clf,scaler.transform(testX),testY,0)

In [ ]:
# in practice we give best result for same time

In [ ]:
# nex step - add synt feature  

In [ ]:
MLData.corr().isFraud

In [ ]:
Ldrop=['oldBalanceDest','newBalanceDest']

In [ ]:
#try add linear probalitics to feature
scaler = StandardScaler()
scaler.fit(X.drop(Ldrop,axis=1))
lr = LogisticRegression(class_weight='balanced', C=0.5, random_state=33)
lr.fit(scaler.transform(trainX.drop(Ldrop,axis=1)),trainY)
trainX1=pd.DataFrame(scaler.transform(trainX.drop(Ldrop,axis=1)))
trainX1['lsynt']=lr.predict_log_proba(trainX1).T[1].reshape(-1,1)
testX1=pd.DataFrame(scaler.transform(testX.drop(Ldrop,axis=1)))
testX1['lsynt']=lr.predict_log_proba(testX1).T[1].reshape(-1,1)
y_pred=lr.predict(scaler.transform(testX.drop(Ldrop,axis=1)))
print  ('Confusion_matrix : ')
print (confusion_matrix(testY,y_pred))

In [ ]:
%%time
# metrics for lsynt feature and  0.5 test size
# Data already scaled
weights = (Y == 0).sum() / (1.0 * (Y == 1).sum())
clf = XGBClassifier( scale_pos_weight = weights, n_jobs = 4, random_state=42)
clf.fit(trainX1, trainY)
presult(clf,testX1,testY,1)

In [ ]:
%%time
#LGBMClassifier
# metrics for 0.3 test size
clf = lgb.LGBMClassifier(n_estimators=100,max_depth=3, n_jobs = 4, random_state=42)
clf.fit(trainX1, trainY)
presult(clf,testX1,testY,0)

Best result for XGB, but not for LGB. Try for 0.3 

In [ ]:
trainX, testX, trainY, testY = train_test_split(X, Y, test_size = 0.3, shuffle=False)

In [ ]:
#try add linear probalitics to feature
scaler = StandardScaler()
scaler.fit(X.drop(Ldrop,axis=1))
lr = LogisticRegression(class_weight='balanced', C=0.5, random_state=33)
lr.fit(scaler.transform(trainX.drop(Ldrop,axis=1)),trainY)
trainX1=pd.DataFrame(scaler.transform(trainX.drop(Ldrop,axis=1)))
trainX1['lsynt']=lr.predict_log_proba(trainX1).T[1].reshape(-1,1)
testX1=pd.DataFrame(scaler.transform(testX.drop(Ldrop,axis=1)))
testX1['lsynt']=lr.predict_log_proba(testX1).T[1].reshape(-1,1)
y_pred=lr.predict(scaler.transform(testX.drop(Ldrop,axis=1)))
print  ('Confusion_matrix : ')
print (confusion_matrix(testY,y_pred))

In [ ]:
%%time
# metrics for lsynt feature and  0.3 test size
# Data already scaled
weights = (Y == 0).sum() / (1.0 * (Y == 1).sum())
clf = XGBClassifier( scale_pos_weight = weights, n_jobs = 4, random_state=42)
clf.fit(trainX1, trainY)
presult(clf,testX1,testY,1)

In [ ]:
# not so good. Try only Scaler

In [ ]:
%%time
scaler = StandardScaler()
scaler.fit(trainX)
# metrics for 0.3 test size
weights = (Y == 0).sum() / (1.0 * (Y == 1).sum())
clf = XGBClassifier( scale_pos_weight = weights, n_jobs = 4, random_state=42)
clf.fit(scaler.transform(trainX), trainY)
presult(clf,scaler.transform(testX),testY,0)

#### IF FN (Fraud) cost >= FP cost * 10  we win , else we loose 

In [ ]:
%%time
#LGBMClassifier
scaler = StandardScaler()
scaler.fit(trainX)
# metrics for 0.3 test size
clf = lgb.LGBMClassifier(n_estimators=100,max_depth=3, n_jobs = 4, random_state=42)
clf.fit(scaler.transform(trainX), trainY)
presult(clf,scaler.transform(testX),testY,0)

## Conclusion


1. I think it's not the right decision to use the mixed sample in time series data. We guess the past based on the knowledge of the future.
2. You should always look at Confusion_matrix in classification problems. Especially in problems of binary classification. My leading synthetic metric for an unbalanced sample is f1-macro
3. For problems of fraud analysis, it is desirable to know the cost of False Positive and False Negative. For example, FP = 1USD and FN = 10USD. This will allow more accurate estimate if the FP and FN are significant
4. I think that the OrigAmntErr, which allows for so precise identification of fraud, is an error of the genius algorithm and will be corrected in the next iterations. The real problems of search for fraud have significantly more noisy data with a worse quality solution
5. Detecting fraud detection is not normal behavior. Normality can be determined on a general background (the amount is more than), but this will probably not be enough. We need a customer profile. For client profile, you need the event geography by the connection's IP address or the city of the transaction execution by the client. You need to connect to the network even without an operation. In the current dataset, unfortunately, the client profile does not have the ability to allocate. There is also no possibility to determine the geography of the operation.
6. LightGBM  demonstrated a higher computational speed than XGBoost with a comparable quality
7. The synthetic frature  the probability of belonging to a class from a linear model does not allow improving the result when the basic characteristics give good quality

#### Thanks to Edgar Alonso Lopez-Rojas for posting this dataset.
#### Thanks to Arjun Joshua for killing features
#### Thanks to Arjun Joshua and Net for perfect data vizualization. These perfect notebooks really help me.

